In [ ]:
from neuprint import Client

with open('auth_token.txt', 'r') as f:
    token = f.read()

c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

In [ ]:
from neuprint import NeuronCriteria as NC
from neuprint import fetch_neurons
import pandas as pd

neuron_df, _ = fetch_neurons(NC(inputRois=['ANm'], outputRois=['ANm'], status='Traced', entryNerve = None, exitNerve=None))

# Only those entirely in the ANm
amn_local = neuron_df[
    neuron_df.apply(
        lambda row: (
            (row['inputRois'] == ['ANm'])
            and (row['outputRois'] == ['ANm'])
            and (row['entryNerve'] == None))
            and (row['exitNerve'] == None),
        axis=1
    )
]

In [ ]:
roi_mesh = c.fetch_roi_mesh('ANm')

In [ ]:
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
from neuprint import fetch_synapses
from neuprint import SynapseCriteria as SC
output_notebook()

NEURON_INDEX = 286

CINNABAR = '#db544b'
LAPIS = '#2d66a5'
BYZANTINE = '#D342BE'
SHAMROCK = '#33A358'

def plot_neurons(source_df, idx : int):

    skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

    skelee.radius = skelee.radius.astype(float)/(20)
    # Join parent/child nodes for plotting as line segments below.
    # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
    skelee = skelee.merge(skelee, 'inner',
                            left_on=['link'],
                            right_on=['rowId'],
                            suffixes=['_child', '_parent'])
    p = figure(
        title = 'Body ID: ' + str(source_df.iloc[idx]['bodyId'])
            + f'(Ach : {round(source_df.iloc[idx]["ntAcetylcholineProb"],2)}, '
            + f'Glu : {round(source_df.iloc[idx]["ntGlutamateProb"],2)}, '
            + f'GABA: {round(source_df.iloc[idx]["ntGabaProb"],2)})',
        match_aspect = True
    )
    p.y_range.flipped = False

    axes = [0,2]
    verts = np.array([[float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])] for row in str(roi_mesh).split('\\n') if row.startswith('v') ])
    speckle_size = 1.0
    fill_color = '#666666'
    fill_alpha = 0.3
    p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = 0.5)


    # Plot skeleton segments (in 2D)
    p.segment(x0='x_child', x1='x_parent',
            y0='z_child', y1='z_parent',
            line_width = 'radius_parent',
            alpha = 0.8,
            color = '#19657F',
            #color='color_child',
            source=skelee)
    
    syns = fetch_synapses(NC(bodyId=source_df.iloc[idx]['bodyId']))
    pre = syns.loc[syns['type'] == 'pre']
    p.scatter(
        pre['x'], pre['z'], size = 3, fill_color=BYZANTINE, fill_alpha = 0.9, line_color = None, line_alpha = 0.0, line_width=0
    )
    post = syns.loc[syns['type'] == 'post']
    p.scatter(
        post['x'], post['z'], size = 3, fill_color=SHAMROCK, fill_alpha = 0.8, line_color=None, line_width = 0,
    )

    p.renderers.insert(0,p.renderers.pop())

    p.y_range = Range1d(0,20000)
    p.x_range = Range1d(18000,30000)
    p.xgrid.visible = False
    p.ygrid.visible = False
    return p

from bokeh.plotting import gridplot

from bokeh.plotting import output_file
from bokeh.io import export_png

#output_file(filename='/Users/stephen/Desktop/Data/SICode/manc/some_neurons.png')
# make a grid

for idx in range(0,len(neuron_df),10):
    grid = gridplot([[plot_neurons(neuron_df,idx + 2*row_num), plot_neurons(neuron_df, idx + 2*row_num+1)] for row_num in range(5)])

    export_png(grid, filename=f"/Users/stephen/Desktop/Data/SICode/manc/Any_AG/some_neurons_{idx}.png")


# Also plot the synapses from the above example
#p.scatter(points['x_post'], points['z_post'], color=points['color'])



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.hist(amn_local[['ntAcetylcholineProb','ntGlutamateProb', 'ntGabaProb']].apply(lambda x : -np.sum(x*np.log2(x)),axis=1), bins = 30)

In [ ]:
neuron_df.loc[neuron_df['bodyId'] == 17897][['ntAcetylcholineProb', 'ntGabaProb','ntGlutamateProb']]

In [ ]:
ambiguous = neuron_df.loc[(neuron_df['predictedNtProb'] < 0.5)]

ambiguous[ambiguous.apply(
        lambda row: (
            (row['inputRois'] == ['ANm'])
            and (row['outputRois'] == ['ANm'])
            and (row['entryNerve'] == None))
            and (row['exitNerve'] == None),
        axis=1
    )
]


for idx in range(0,len(ambiguous),10):
    grid = gridplot([[plot_neurons(ambiguous,idx + 2*row_num), plot_neurons(ambiguous, idx + 2*row_num+1)] for row_num in range(5)])

    export_png(grid, filename=f"/Users/stephen/Desktop/Data/SICode/manc/ambiguous_neurons/some_neurons_{idx}.png")

In [ ]:
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
from neuprint import fetch_synapses
from neuprint import SynapseCriteria as SC

syns = fetch_synapses(neuron_df.iloc[500]['bodyId'])

syns.loc[syns['type'] == 'post']

In [ ]:
axes = [0,2]
verts = np.array([[float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])] for row in str(roi_mesh).split('\\n') if row.startswith('v') ])
speckle_size = 0.5
fill_color = '#AAAAAA',
fill_alpha = 0.4
p.scatter(verts[:,0],verts[:,1],size=speckle_size,color=fill_color,alpha=fill_alpha,line_alpha=0.0)

p.renderers.insert(0,p.renderers.pop())

In [ ]:
from scipy import datasets

f = datasets.face()

In [ ]:
neuron_df[neuron_df['bodyId'] == 14371]